<a href="https://colab.research.google.com/github/festeban11/Minor-Machine-Learning/blob/main/ProyectoFinal-HumanActivityRecognition-Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final – Human Activity Recognition

**Nombre:** Fernando Ibacache Carrión

**1. Problema:** Clasificar actividades humanas a partir de datos 
provenientes de sensores.

**2. Hipótesis:** Es posible reconocer la actividad humana a partir de sensores con métricas de rendimiento superior de 95%.

**3. Solución:** Se propone una investigación e implementación de tres modelos de clasificación; Support Vector Machine (SVM), Random Forest y Extreme Gradient Boosting (XGBoost). Todos los modelos usarán las mismas validaciones, en problemas de clasificación las métricas de rendimiento más utilizadas son: Accuracy (ACC), Precision (P), Recall (R), F1 score(F1) y Area Under Crurve (AUC).

###Entrega 01 - OBTAIN

In [110]:
import numpy as np
import pandas as pd

In [111]:
columns = ['user','activity','timestamp', 'x-accel', 'y-accel', 'z-accel']
df = pd.read_csv('drive/MyDrive/WISDM_at_v2.0_raw.txt', header = None, names = columns, low_memory=False)
df.head()

,user,activity,timestamp,x-accel,y-accel,z-accel
0,1679,Walking,1370520469556,0.294132,-0.635605,-0.22693644;
1,1679,Walking,1370520469606,-0.499688,-0.604451,-0.22602014;
2,1679,Walking,1370520469656,-2.178345,0.713491,0.37201694;
3,1679,Walking,1370520469706,-2.797763,1.354899,-0.27763826;
4,1679,Walking,1370520469756,-2.167961,-1.327716,-0.5549711;


*   user: código asignado al usuario
*   activity: actividad realizada (Walking, Jogging, Sitting, Standing,Upstairs, Downstairs)
* timestamp: marca de tiempo (tiempo en milisegundos a través del tiempo unix)
* x-accel: aceleración en la dirección x medida por el acelerómetro del teléfono android en m/(s^2).
* y-accel: aceleración en la dirección y medida por el acelerómetro del teléfono android en m/(s^2).
* z-accel: aceleración en la dirección z medida por el acelerómetro del teléfono android en m/(s^2).



In [112]:
df.shape

(3005411, 6)

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005411 entries, 0 to 3005410
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user       int64  
 1   activity   object 
 2   timestamp  object 
 3   x-accel    float64
 4   y-accel    float64
 5   z-accel    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 137.6+ MB


###Entrega 02 - SCRUB

Eliminar filas con datos faltantes



In [114]:
#Se revisa si existen datos faltantes con la funcion isnull()
df.isnull().sum()

user             0
activity     24646
timestamp        0
x-accel      24646
y-accel      24646
z-accel      24646
dtype: int64

In [115]:
#Se eliminan las filas con datos faltantes usando la funcion dropna()
df=df.dropna()

Agregar al DataFrame los nombres de las columnas \\
**Se realizó en la etapa OBTAIN**

Visualizar las 5 primeras filas del DataFrame

In [116]:
df.head()

,user,activity,timestamp,x-accel,y-accel,z-accel
0,1679,Walking,1370520469556,0.294132,-0.635605,-0.22693644;
1,1679,Walking,1370520469606,-0.499688,-0.604451,-0.22602014;
2,1679,Walking,1370520469656,-2.178345,0.713491,0.37201694;
3,1679,Walking,1370520469706,-2.797763,1.354899,-0.27763826;
4,1679,Walking,1370520469756,-2.167961,-1.327716,-0.5549711;


Buscar datos nulos y eliminarlos si es que existen

In [117]:
df.isnull().sum()

user         0
activity     0
timestamp    0
x-accel      0
y-accel      0
z-accel      0
dtype: int64

Visualizar los tipo de datos a los que pertenece cada columna (.info())

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2980765 entries, 0 to 3005410
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user       int64  
 1   activity   object 
 2   timestamp  object 
 3   x-accel    float64
 4   y-accel    float64
 5   z-accel    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 159.2+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación

In [119]:
#Se observo que la columna z-accel tiene datos de tipo object.
#Los valores de la columna z-accel contienen el simbolo de punto y coma.
#Se eliminara el simbolo punto y coma de cada valor usando la funcion split, 
#luego seran agregados nuevamente al dataframe

columnZ = df['z-accel']
zValuesClen = []
for zValues in columnZ:
  zValuesClen.append(str(zValues).split(';')[0])
df['z-accel']=zValuesClen
df['z-accel']

0          -0.22693644
1          -0.22602014
2           0.37201694
3          -0.27763826
4           -0.5549711
              ...     
3005406      3.1362782
3005407       9.152567
3005408      10.953622
3005409       5.588778
3005410    -0.23590942
Name: z-accel, Length: 2980765, dtype: object

In [120]:
#Se convierten los valores de la columna z-accel a float
df['z-accel'] = df['z-accel'].astype('float')

Imprimir los tipos de datos de las variables del DataFrame

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2980765 entries, 0 to 3005410
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user       int64  
 1   activity   object 
 2   timestamp  object 
 3   x-accel    float64
 4   y-accel    float64
 5   z-accel    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 159.2+ MB


Eliminar las columnas que no se utilizaran para la clasificación

In [122]:
#No se usaran las columnas 'user y 'timestamp'
df = df.drop(['user', 'timestamp'], axis = 1).copy()
df.head()

,activity,x-accel,y-accel,z-accel
0,Walking,0.294132,-0.635605,-0.226936
1,Walking,-0.499688,-0.604451,-0.226020
2,Walking,-2.178345,0.713491,0.372017
3,Walking,-2.797763,1.354899,-0.277638
4,Walking,-2.167961,-1.327716,-0.554971


Utilizar el comando value_counts para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar

In [123]:
df['activity'].value_counts()

Walking      1255923
Sitting       663706
Jogging       438871
Standing      288873
LyingDown     275967
Stairs         57425
Name: activity, dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias.

In [126]:
#Se utiliza la actividad "Stairs" debido que tiene el menor números de instancias en comparacion a las demas actividades
Walking = df[df['activity']=='Walking'].head(57425).copy()
Sitting = df[df['activity']=='Sitting'].head(57425).copy()
Jogging = df[df['activity']=='Jogging'].head(57425).copy()
Standing  = df[df['activity']=='Standing'].head(57425).copy()
LyingDown = df[df['activity']=='LyingDown'].head(57425).copy()
Stairs = df[df['activity']=='Stairs'].copy()
balancedDf = pd.DataFrame()
balancedDf = balancedDf.append([Walking, Sitting, Jogging, Standing, LyingDown, Stairs])

Establecer el número de instancias y dimensiones del data frame resultante (shape)

In [127]:
balancedDf.shape

(344550, 4)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. (value_counts)

In [128]:
balancedDf['activity'].value_counts()

Walking      57425
Sitting      57425
Jogging      57425
Standing     57425
LyingDown    57425
Stairs       57425
Name: activity, dtype: int64

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden (head)

In [129]:
balancedDf.head()

,activity,x-accel,y-accel,z-accel
0,Walking,0.294132,-0.635605,-0.226936
1,Walking,-0.499688,-0.604451,-0.226020
2,Walking,-2.178345,0.713491,0.372017
3,Walking,-2.797763,1.354899,-0.277638
4,Walking,-2.167961,-1.327716,-0.554971


Separar las variables predictoras (X) de las categorias o etiquetas (y)

In [130]:
from sklearn.preprocessing import LabelEncoder

In [131]:
label = LabelEncoder()
balancedDf['label'] = label.fit_transform(balancedDf['activity'])
balancedDf.head()

,activity,x-accel,y-accel,z-accel,label
0,Walking,0.294132,-0.635605,-0.226936,5
1,Walking,-0.499688,-0.604451,-0.226020,5
2,Walking,-2.178345,0.713491,0.372017,5
3,Walking,-2.797763,1.354899,-0.277638,5
4,Walking,-2.167961,-1.327716,-0.554971,5


In [132]:
X = balancedDf[['x-accel', 'y-accel', 'z-accel']]
y = balancedDf['label']

Estandarizar los datos numericos (StandardScaler)

In [133]:
from sklearn.preprocessing import StandardScaler

In [99]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

,x-accel,y-accel,z-accel,label
0,-0.001704,0.001704,-0.001704,5
1,-0.001704,0.001704,-0.001704,5
2,-0.001704,0.001704,-0.001704,5
3,-0.001704,0.001704,-0.001704,5
4,-0.001704,0.001704,-0.001704,5


Como último paso debe reconstruir el dataframe pegando las columnas de los variables predicadores (X) y las categorías (y)

In [134]:
scaledX = pd.DataFrame(data = X, columns = ['x-accel', 'y-accel', 'z-accel'])
scaledX['label'] = y.values
scaledX.head()

,x-accel,y-accel,z-accel,label
0,0.294132,-0.635605,-0.226936,5
1,-0.499688,-0.604451,-0.226020,5
2,-2.178345,0.713491,0.372017,5
3,-2.797763,1.354899,-0.277638,5
4,-2.167961,-1.327716,-0.554971,5
